In [66]:
import os
import sys
import shutil
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

from Import_csv import import_csv
imported_csv_path = import_csv()

df = pd.read_csv(imported_csv_path)

Downloading...
From (original): https://drive.google.com/uc?id=1vdNaWcJgMzUnlaDVA4FuZRFYinO8UsPQ
From (redirected): https://drive.google.com/uc?id=1vdNaWcJgMzUnlaDVA4FuZRFYinO8UsPQ&confirm=t&uuid=5b1495ea-58ac-4090-9955-3e1273a5e959
To: c:\Users\33679\OneDrive\Bureau\brief predict pret\Model_pret\Dataset\SBAnational.csv
100%|██████████| 179M/179M [00:02<00:00, 84.7MB/s] 
C:\Users\33679\AppData\Local\Temp\ipykernel_33240\65876671.py:31: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [67]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[4014374000, 3242005009, 8556154007, 336125600..."
1,Name,object,14,0.00,779583,"[PETE AND DAVE, LTD., PAJARO VALLEY CONSTRUCTI..."
2,City,object,30,0.00,32581,"[LOS ANGELES, Semmes, PAINESVILLE, HOUSTON, CA..."
3,State,object,14,0.00,51,"[CA, IL, VA, OR]"
4,Zip,int64,0,0.00,33611,"[67654, 25978, 72396, 77904, 75087]"
5,Bank,object,1559,0.17,5802,"[CITIZENS BANK NATL ASSOC, PNC BANK, NATIONAL ..."
6,BankState,object,1566,0.17,56,"[UT, AL]"
7,NAICS,int64,0,0.00,1312,"[0, 445299, 621210, 238990, 442110]"
8,ApprovalDate,object,0,0.00,9859,"[2-Apr-98, 1-Oct-07, 18-Mar-97, 3-Aug-99, 25-A..."
9,ApprovalFY,object,0,0.00,70,"[2002, 2013, 2010, 2005, 1993]"


In [68]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [69]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [70]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)].astype(object)
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [71]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [72]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [73]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [74]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [75]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

df['Franchise'] = df['Franchise'].astype(object)
df['LowDoc'] = df['LowDoc'].astype(object)
df['RevLineCr'] = df['RevLineCr'].astype(object)
df['NewExist'] = df['NewExist'].astype(object)
df['ApprovalFY'] = df['ApprovalFY'].astype(object)
df['ApprovalDate'] = df['ApprovalDate'].astype(object)
df['NAICS'] = df['NAICS'].astype(object)
df['BankState'] = df['BankState'].astype(object)
df['Bank'] = df['Bank'].astype(object)
df['State'] = df['State'].astype(object)
df['City'] = df['City'].astype(object)


In [76]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [77]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [78]:
df.shape

(675026, 13)

In [79]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[72, 42, 44-45, 31-33]"
1,Term,object,0,0.0,377,"[84, 142, 120, 60]"
2,NoEmp,object,0,0.0,499,"[3, 4, 1, 6]"
3,NewExist,object,0,0.0,2,[1.0]
4,CreateJob,object,0,0.0,216,"[0, 3, 6]"
5,RetainedJob,object,0,0.0,323,"[14, 10, 0, 23, 4]"
6,UrbanRural,object,0,0.0,3,[1]
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,"[N, Y]"
9,GrAppv,int32,0,0.0,16880,"[715000, 100000, 31000, 50000, 152400]"


In [80]:
df_to_csv(df, folder_path, 'SBA_Cleaned_0.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset\SBA_Cleaned_0.csv


In [81]:
df = pd.read_csv(imported_csv_path)

C:\Users\33679\AppData\Local\Temp\ipykernel_33240\3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [82]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [83]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [84]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [85]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [86]:
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [87]:
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [88]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [89]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

df['Franchise'] = df['Franchise'].astype(object)
df['LowDoc'] = df['LowDoc'].astype(object)
df['RevLineCr'] = df['RevLineCr'].astype(object)
df['NewExist'] = df['NewExist'].astype(object)
df['ApprovalFY'] = df['ApprovalFY'].astype(object)
df['ApprovalDate'] = df['ApprovalDate'].astype(object)
df['NAICS'] = df['NAICS'].astype(object)
df['BankState'] = df['BankState'].astype(object)
df['Bank'] = df['Bank'].astype(object)
df['State'] = df['State'].astype(object)
df['City'] = df['City'].astype(object)


In [90]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [91]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [92]:
df.shape

(460816, 13)

In [93]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[62, 48-49, 54, 44-45, 56]"
1,Term,int64,0,0.0,373,"[6, 44, 84, 281, 300]"
2,NoEmp,int64,0,0.0,432,"[6, 4, 2, 5]"
3,NewExist,object,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,192,"[14, 0]"
5,RetainedJob,int64,0,0.0,276,"[0, 3]"
6,UrbanRural,int64,0,0.0,3,"[1, 0]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int32,0,0.0,12824,"[10000, 163000, 250000, 75000]"


In [94]:
df_to_csv(df, folder_path, 'SBA_Cleaned_1.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset\SBA_Cleaned_1.csv


In [95]:
df = pd.read_csv(imported_csv_path)

C:\Users\33679\AppData\Local\Temp\ipykernel_33240\3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [96]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [97]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [98]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [99]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [100]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [101]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [102]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [103]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [104]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [105]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [106]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [107]:
df.shape

(675026, 13)

In [108]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[31-33, 42, 72, 54]"
1,Term,int64,0,0.0,377,"[32, 84, 60, 120]"
2,NoEmp,int64,0,0.0,499,"[1, 2, 5, 8, 0]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[0, 2]"
5,RetainedJob,int64,0,0.0,323,"[0, 1, 7]"
6,UrbanRural,int64,0,0.0,3,"[1, 2, 0]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int32,0,0.0,16880,"[90000, 10000, 30000, 150000, 1391000]"


In [109]:
df_to_csv(df, folder_path, 'SBA_Cleaned_2.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset\SBA_Cleaned_2.csv


In [110]:
df = pd.read_csv(imported_csv_path)

C:\Users\33679\AppData\Local\Temp\ipykernel_33240\3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [111]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[7266404007, 2560913009, 1078445010, 612894300..."
1,Name,object,14,0.00,779583,"[Seifert's High 5 Sports LLC, DIVERSIFIED SERV..."
2,City,object,30,0.00,32581,"[MADISON, LACEY, Las Vegas, WATERTOWN, SIMSBORO]"
3,State,object,14,0.00,51,"[KS, GA, TX, CO, CA]"
4,Zip,int64,0,0.00,33611,"[43251, 47715, 80919, 32757, 73601]"
5,Bank,object,1559,0.17,5802,"[BBCN BANK, MACHIAS SAVINGS BANK, CSRA LOCAL D..."
6,BankState,object,1566,0.17,56,"[IN, VA, WI, CA, PA]"
7,NAICS,int64,0,0.00,1312,"[0, 812320, 541512]"
8,ApprovalDate,object,0,0.00,9859,"[2-Apr-99, 21-Jul-93, 12-Dec-07, 4-May-04, 27-..."
9,ApprovalFY,object,0,0.00,70,"[1996, 2000, 2008, 1993]"


In [112]:
df = df.drop_duplicates(subset='LoanNr_ChkDgt')

In [113]:
df['NewExist'] = df['NewExist'].isin({1.0, 2.0}).astype(object)

In [114]:
is_valide = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].isin(is_valide).astype(object)

In [115]:
is_valide = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].isin(is_valide).astype(object)

In [116]:
df = df.dropna(subset='NAICS')
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [117]:
mapping = {0: 'unknown', 1: 'urban', 2: 'rural'}
df['UrbanRural'] = df['UrbanRural'].map(mapping).astype(object)

In [118]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1).astype(object)
df = df.drop('FranchiseCode', axis=1)

In [119]:
df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'], format='%d-%b-%y').dt.strftime('%d-%m-%y').astype(object)

In [120]:
df['ApprovalDate'].unique()

array(['28-02-97', '07-02-06', '25-03-97', ..., '25-02-97', '26-02-97',
       '27-02-97'], dtype=object)

In [121]:
df['ApprovalFY'] = df['ApprovalFY'].astype(str).apply(lambda x: '20' + x[-2:] if int(x[-2:]) >= 0 and int(x[-2:]) <= 27 else '19' + x[-2:]).astype(object)
df['ApprovalFY'].unique()

array(['1997', '2006', '1998', '1999', '2000', '2001', '2003', '2004',
       '2005', '1984', '2007', '1987', '2008', '1988', '2009', '1989',
       '1990', '2010', '1991', '2011', '1992', '2002', '2012', '1993',
       '2013', '1994', '2014', '1995', '1996'], dtype=object)

In [122]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [123]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & (df['ChgOffPrinGr'] != 0.0)]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [124]:
samples_bad_lol = df[df['ChgOffDate'].notna()]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [125]:
df['MIS_Status'].value_counts()

MIS_Status
P I F     550334
CHGOFF    145335
Name: count, dtype: int64

In [126]:
df['Approve'] = df['MIS_Status'].apply(lambda x: 1 if x == 'P I F' else 0).astype(bool)
df.drop(['MIS_Status'], axis=1, inplace=True)

In [127]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Zip', 'LoanNr_ChkDgt']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df = df.dropna()

In [128]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,Name,object,0,0.0,611131,"[IMAGINE THAT!, DOUBLE CROSS JEWELRY, Law Offi..."
1,City,object,0,0.0,28860,"[WEST SPRINGFIELD (TOWN OF), STILLWATER, SAINT..."
2,State,object,0,0.0,51,"[NC, ID, KS, RI, WI]"
3,Bank,object,0,0.0,5221,"[BBCN BANK, STEARNS BK NATL ASSOC, WELLS FARGO..."
4,BankState,object,0,0.0,55,"[NC, RI, IL, CA]"
5,NAICS,object,0,0.0,20,"[62, 54, 81, 48-49, 23]"
6,ApprovalDate,object,0,0.0,7367,"[01-04-10, 30-10-01, 14-07-95, 24-04-06, 31-05..."
7,ApprovalFY,object,0,0.0,29,"[1997, 2005, 2006, 2003]"
8,Term,int64,0,0.0,378,"[60, 228, 84, 59]"
9,NoEmp,int64,0,0.0,502,"[1, 3, 30]"


In [129]:
df.shape

(696572, 20)

In [130]:
df_to_csv(df, folder_path, 'SBA_Cleaned_Pure.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset\SBA_Cleaned_Pure.csv
